In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import re
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from uuid import uuid4

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder = os.path.expanduser('~/Desktop/Machine Learning/data_ML_Project')
file_path = os.path.join(folder, 'earnings_calls.parquet')  # No extension

if os.path.exists(file_path):
    data = pd.read_parquet(file_path)
else:
    print("File still not found!")

In [3]:
data.head()

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
0,3285682,0,"Good morning, everyone, and welcome to the LXI...",2020-11-23,323562,LXI REIT plc,Presentation Operator Message
1,50630,5,"Thanks, Bob. In summary 2009 was a pivotal yea...",2010-02-18,063083,Endologix LLC,Presenter Speech
2,840499,103,"Just one thing left to ask, actually. If we ta...",2015-07-28,002410,BP p.l.c.,Question
3,47300,58,"I said, I think it certainly has been a rare o...",2010-01-27,009317,SEI Investments Company,Answer
4,49778,1,<strong>Operator</strong>\nLadies and gentleme...,2010-02-12,176660,3SBio Inc.,Presentation Section


In [4]:
data.sort_values(by='mostimportantdateutc',inplace=True)

data.sort_values(by=['mostimportantdateutc', 'componentorder'], ascending=[False, True],inplace=True)

data

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
88038,2164922,0,"Ladies and gentlemen, thank you for standing b...",2020-12-30,161925,China Finance Online Co. Limited,Presentation Operator Message
958919,2164922,1,"Thank you. Thank you, operator. Welcome to Chi...",2020-12-30,161925,China Finance Online Co. Limited,Presenter Speech
958914,2164922,2,[Foreign Language],2020-12-30,161925,China Finance Online Co. Limited,Presenter Speech
958909,2164922,3,"Good morning, and good evening. Thank you for ...",2020-12-30,161925,China Finance Online Co. Limited,Presenter Speech
958904,2164922,4,[Foreign Language],2020-12-30,161925,China Finance Online Co. Limited,Presenter Speech
...,...,...,...,...,...,...,...
4581352,46682,84,And then I did want to ask Tom about Cisco. Wh...,2010-01-04,030247,Viasystems Corporation,Question
41828,46682,85,We don't typically comment on any specific cus...,2010-01-04,030247,Viasystems Corporation,Answer
4020336,46682,86,"And Mr. Burger, we have no further questions.",2010-01-04,030247,Viasystems Corporation,Question and Answer Operator Message
41822,46682,87,"Well, thank you, everyone for attending, we ap...",2010-01-04,030247,Viasystems Corporation,Answer


In [10]:
# Group and merge componenttext by transcriptid
grouped_data = data.groupby('transcriptid')['componenttext'].apply(lambda texts: ' '.join(texts)).reset_index()

# Rename the column properly on the new DataFrame
grouped_data.rename(columns={'componenttext': 'full_transcript'}, inplace=True)


In [11]:
grouped_data

,transcriptid,full_transcript
0,15674,"Good morning, and welcome to Pactiv’s fourth q..."
1,45053,"Ladies and gentlemen, thank you for standing b..."
2,45107,"Ladies and gentlemen, thank you for standing b..."
3,45200,"Good morning, my name is Kim, and I will be yo..."
4,45212,"Good morning, ladies and gentlemen, and welcom..."
...,...,...
215308,3328351,Good afternoon. My name is Tom and I will be y...
215309,3328562,"Good morning. My name is Dorothy, and I will b..."
215310,3341730,Greetings and welcome to miRagen Therapeutics ...
215311,3342207,"Good morning, everyone, and welcome to Encompa..."


In [160]:
def extract_number_context(text, words_before=4, words_after=4):
    # Only match numbers followed by a percent sign
    pattern = re.compile(r'(\$?\d[\d,.]*\s*%)', flags=re.IGNORECASE)
    
    # Split text into words once
    words = text.split()
    word_positions = {}
    
    # Create a mapping of character positions to word indices
    char_pos = 0
    for i, word in enumerate(words):
        word_positions[char_pos] = i
        char_pos += len(word) + 1  # +1 for the space
    
    results = []
    for match in pattern.finditer(text):
        num_str = match.group(1).strip()

        # Normalize for safety in checking plain numeric values
        numeric_only = num_str.replace(',', '').replace('$', '').replace('%', '').strip().lower()

        # Skip if it's a year (2000–2025), just in case something weird slips through
        if numeric_only.isdigit():
            year = int(numeric_only)
            if 2000 <= year <= 2025:
                continue
        
        # Find the closest word position before the match
        match_start = match.start()
        closest_pos = max(pos for pos in word_positions if pos <= match_start)
        word_idx = word_positions[closest_pos]
        
        # Get context words
        context_before = " ".join(words[max(0, word_idx - words_before):word_idx])
        context_after = " ".join(words[word_idx:word_idx + words_after])
        
        results.append({
            "Context_Before": context_before,
            "Number": num_str,
            "Context_After": context_after
        })
    
    return pd.DataFrame(results)


In [167]:
results = []

for _, row in grouped_data.iterrows():
    df = extract_number_context(row["full_transcript"])
    df["transcriptid"] = row["transcriptid"]
    results.append(df)

results_df = pd.concat(results, ignore_index=True)


In [171]:
results_df.to_parquet('results_inter.parquet', index=False)

In [170]:
def extract_financial_data(df):
    """
    Extract important financial information from the dataframe.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with Context_Before, Number, Context_After columns
    
    Returns:
    pandas.DataFrame: Filtered DataFrame containing only relevant financial information
    """
    # Define patterns to identify important financial data
    financial_keywords = [
        # Earnings related
        'earnings', 'profit', 'revenue', 'income', 'ebitda', 'eps', 'per share',
        # Growth metrics
        'growth', 'increase', 'decrease', 'up', 'down',
        # Financial ratios and metrics
        'margin', 'ratio', 'return on', 'roe', 'roa', 'roce', 'operating margin', 'gross margin',
        'ebitda', 'net debt', 'leverage', 'cap', 'debt to', 'coverage',
        # Guidance/Outlook
        'guidance', 'outlook', 'forecast', 'projection', 'estimate', 'anticipated',
        # Market indicators
        'market share', 'pricing', 'volume', 'sales', 'demand',
        # Cash flow related
        'free cash flow', 'cash flow', 'capex', 'capital expenditure', 'working capital',
        # Cost related
        'cost', 'expense', 'sg&a', 'restructuring', 'raw material', 'resin cost',
        # Tax related
        'tax rate', 'effective tax', 'cash tax',
        # Other financial terms
        'million', 'billion', 'basis points', 'liquidity', 'debt', 'contribution',
        'cash', 'acquisition', 'depreciation', 'amortization', 'interest expense'
    ]
    
    # Create filtering conditions
    def has_financial_context(row):
        context_before = row['Context_Before'].lower()
        context_after = row['Context_After'].lower()
        combined_context = context_before + " " + context_after
        return any(keyword in combined_context for keyword in financial_keywords)
    
    # Filter rows with financial relevance
    financial_df = df[df.apply(has_financial_context, axis=1)].copy()
    
    # Add a column to categorize the type of financial data
    def categorize_financial_data(row):
        context = (row['Context_Before'] + " " + row['Context_After']).lower()
        
        if any(term in context for term in ['earnings per share', 'eps', 'per share']):
            return 'Earnings Per Share'
        elif any(term in context for term in ['revenue']):
            return 'Revenue'
        elif any(term in context for term in ['pricing', 'price']):
            return 'Pricing'
        elif any(term in context for term in ['volume', 'units']):
            return 'Volume'
        else:
            return 'Other Financial Data'
    
    financial_df['Data_Category'] = financial_df.apply(categorize_financial_data, axis=1)
    
    # Remove "Other Financial Data"
    financial_df = financial_df[financial_df['Data_Category'] != 'Other Financial Data']
    
    # Add + or - sign based on sentiment in context
    def add_sign(row):
        context = (row['Context_Before'] + " " + row['Context_After']).lower()
        number = row['Number']
        
        # Skip if it already starts with a sign
        if number.startswith(('+', '-')):
            return number
        
        if any(word in context for word in ['decline', 'decrease', 'down', 'dropped', 'fell']):
            return '-' + number
        elif any(word in context for word in ['increase', 'up', 'grew', 'rose', 'jumped']):
            return '+' + number
        else:
            return number

    financial_df['Number'] = financial_df.apply(add_sign, axis=1)

    # Reorder columns
    columns_order = ['Data_Category', 'Number', 'Context_Before', 'Context_After', 'transcriptid']
    return financial_df[columns_order]


In [172]:
financial_df = extract_financial_data(results_df)

In [178]:
financial_df.head(20)

,Data_Category,Number,Context_Before,Context_After,transcriptid
0,Pricing,+6%,Our pricing was up,"6% year-over-year, and our",15674
1,Volume,-4%,volume was down about,4%. Both segments posted,15674
2,Volume,-5%,in sales as a,5% volume decline in,15674
3,Pricing,6%,"unfavorable foreign exchange offset,",6% favorable year-over-year pricing.,15674
5,Pricing,6%,the strength of 6%,"higher pricing, but offset",15674
6,Volume,-4%,but offset of 4%,"volume decline, and 1%",15674
7,Volume,-1%,"volume decline, and 1%",unfavorable foreign exchange. Earnings,15674
8,Earnings Per Share,+18%,year-over-year EPS increase of,"18%, primarily reflect an",15674
21,Volume,-3%,outlook incorporates a 3%,"to 4% volume decline,",15674
22,Volume,-4%,a 3% to 4%,"volume decline, and a",15674


In [181]:
len(financial_df)

753711

In [185]:
def clean_number(val):
    try:
        # Remove percent signs and whitespace
        val = str(val).strip().replace('%', '')
        # Fix stray dots like '1.8.'
        if val.endswith('.'):
            val = val[:-1]
        # Handle plus and minus signs properly
        if val.startswith('+'):
            val = val[1:]
        # Convert to float
        return float(val)
    except:
        return np.nan  # if it can't be converted

# Assuming financial_df is your DataFrame with the financial metrics
financial_df['Number'] = financial_df['Number'].apply(clean_number)

# Filter for the relevant categories
relevant_categories = ['Revenue', 'Pricing', 'Volume', 'Earnings Per Share']
filtered_df = financial_df[financial_df['Data_Category'].isin(relevant_categories)].copy()

# Calculate median values grouped by transcript ID and category
average_metrics = (
    filtered_df
    .groupby(['transcriptid', 'Data_Category'])['Number']
    .median()
    .unstack(fill_value=np.nan)  # Using np.nan instead of 0 to avoid confusing missing data with zero
    .reset_index()
)

average_metrics = average_metrics.fillna(0)

average_metrics


Data_Category,transcriptid,Earnings Per Share,Pricing,Revenue,Volume
0,15674,18.0,6.0,0.0,-4.0
1,45053,0.0,0.0,0.0,18.0
2,45212,0.0,0.0,10.0,0.0
3,45220,0.0,1.0,0.0,-5.6
4,45248,0.0,0.0,20.5,13.0
...,...,...,...,...,...
150145,3317762,0.0,0.0,26.0,0.0
150146,3328351,0.0,8.5,17.0,70.0
150147,3328562,0.0,0.0,2.0,-2.6
150148,3342207,7.4,-0.5,7.0,0.0


In [186]:
transcript_info = data[['transcriptid', 'gvkey', 'mostimportantdateutc']].drop_duplicates()

# Merge the average metrics with the transcript info
average_metrics_with_info = average_metrics.merge(
    transcript_info,
    on='transcriptid',
    how='left'
)

# Convert date to datetime format (optional)
average_metrics_with_info['mostimportantdateutc'] = pd.to_datetime(average_metrics_with_info['mostimportantdateutc'])

average_metrics_with_info

,transcriptid,Earnings Per Share,Pricing,Revenue,Volume,gvkey,mostimportantdateutc
0,15674,18.0,6.0,0.0,-4.0,125533,2010-01-27
1,45053,0.0,0.0,0.0,18.0,008577,2010-01-05
2,45212,0.0,0.0,10.0,0.0,010353,2010-01-06
3,45220,0.0,1.0,0.0,-5.6,008902,2010-01-06
4,45248,0.0,0.0,20.5,13.0,011600,2010-01-06
...,...,...,...,...,...,...,...
150744,3317762,0.0,0.0,26.0,0.0,002615,2019-04-30
150745,3328351,0.0,8.5,17.0,70.0,165746,2017-05-04
150746,3328562,0.0,0.0,2.0,-2.6,268208,2019-10-31
150747,3342207,7.4,-0.5,7.0,0.0,012589,2020-02-07


In [188]:
average_metrics_with_info.to_csv('results_EARNINGS.csv', index=False)